In [1]:
from bs4 import BeautifulSoup as bs
import random
import time
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC

In [2]:
browser = uc.Chrome(driver_executable_path='../chromedriver-linux64/chromedriver')

In [3]:
df = pd.read_csv('data1.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   path       141 non-null    object
 1   tipo       141 non-null    object
 2   estado     141 non-null    object
 3   ubicacion  141 non-null    object
dtypes: object(4)
memory usage: 4.5+ KB


In [5]:
df['path'].value_counts()

path
4a8f-ba4b-18fa5dd-894c5d123024-765f     2
6bd5-a4c8-1bfc2b17-3c2284aa93f5-3425    2
2826-9d40-74f94578-153348823f72-30a7    2
c356-9bd3-ad48bf6c-5f2d1d014225-308c    2
2a35-9bb6-111c69a1-1d8dd4f1069a-3600    2
                                       ..
171a-bd69-a424b003-50d6c9afb973-33bc    1
7740-b75c-93b4aa1d-b43d08fd7331-32d6    1
1ff7-9ad0-29f49d26-165ad78f1f1d-3c34    1
4afc-8c75-2d34c208-b6e46b815570-364d    1
f00c-ab1a-18ff0f4-af52b79b3e4c-7663     1
Name: count, Length: 136, dtype: int64

In [5]:
df.drop_duplicates('path',ignore_index=True,inplace=True)
df.duplicated('path')

0      False
1      False
2      False
3      False
4      False
       ...  
131    False
132    False
133    False
134    False
135    False
Length: 136, dtype: bool

In [71]:
df.shape

(136, 4)

In [20]:
print(str(df.iloc[0,0]).strip("''[]"))


2b72-82fe-18fc293-722dc1ef005c-7129


In [6]:
path = 'https://www.puntopropiedad.com/inmueble/'
url = str(df.iloc[0,0]).strip("''[]")
browser.get(path+url)

In [7]:
html = browser.page_source
soup= bs(html,'html')

In [9]:
temp = soup.find('div',{'class':'priceChars'})
temp

<div class="priceChars">
<div class="price">
<h2>
                            
                                
                                290.000.000 COP$
                            
                        </h2>
</div>
<ul class="details_list">
<li class="dimensions">75m2</li>
<li class="bedrooms">2 Habitaciones</li>
<li class="bathrooms">2 Baños</li>
</ul>
</div>

In [10]:
p = temp.find('div',{'class':'price'})
patron = r'[\d.]+'
condi = re.findall(patron,p.text)
t=''.join(condi)

In [11]:
a=temp.find('li',{'class':'bedrooms'}).text
b = temp.find('li',{'class':'bathrooms'}).text
c = temp.find('li',{'class':'dimensions'}).text
clave = ['url','habitacion','baño','dimension','precio']
valor = [url,a[0],b[0],c,t]

In [12]:
li_elements=soup.find('div',{'class':'dropdown-list open'}).find_all('li',{'class':'tick'})

In [13]:
for li in li_elements:
    if 'Estrato' in li.text:
        # Usa una expresión regular para extraer el número de estrato
        estrato_match = re.search(r'Estrato:\s*(\d+)', li.text)
        if estrato_match:
            estrato_value = estrato_match.group(1)
            valor.append(estrato_value)
            clave.append('estrato')
            

In [23]:
latitud= soup.find('div',{'class':'dropdown-list light open map-section'}).find('button').get('data-x')
longitud= soup.find('div',{'class':'dropdown-list light open map-section'}).find('button').get('data-y')
valor.append(latitud)
clave.append('latitud')
valor.append(longitud)
clave.append('longitud')


In [24]:
data = pd.DataFrame(data=[valor],columns=clave)
data

,url,habitacion,baño,dimension,precio,estrato,latitud,longitud
0,2b72-82fe-18fc293-722dc1ef005c-7129,2,2,75m2,290.000.000,5,5.532431,-73.361601


In [49]:
data1 = data.copy()

## Automatizando para datos de punto_propiedad

In [75]:
path = 'https://www.puntopropiedad.com/inmueble/'
for i in range(77,df.shape[0]):
    url = str(df.iloc[i,0]).strip("''[]")
    browser.get(path+url)
    time.sleep(random.randint(10,12))
    html = browser.page_source
    soup = bs(html,'html')                             
    temp = soup.find('div',{'class':'priceChars'})
    p = temp.find('div',{'class':'price'})
    patron = r'[\d.]+'
    condi = re.findall(patron,p.text)
    p =''.join(condi)
    try:
        a=temp.find('li',{'class':'bedrooms'}).text
    except AttributeError:
        a = 'NAN' 
    try:
        b = temp.find('li',{'class':'bathrooms'}).text
    except AttributeError:
        b = 'NAN' 
    try:
        c = temp.find('li',{'class':'dimensions'}).text
    except:
        c= 'NAN'
    clave = ['url','habitacion','baño','dimension','precio']
    valor = [url,a[0],b[0],c,p]
    # vallamos a por el estrato 
    c = True
    try:
        li_elements=soup.find('div',{'class':'dropdown-list open'}).find_all('li',{'class':'tick'})
    except:
        c = False
        
    if c: 
        for li in li_elements:
            if 'Estrato' in li.text:
        # Usa una expresión regular para extraer el número de estrato
                estrato_match = re.search(r'Estrato:\s*(\d+)', li.text)
                if estrato_match:
                    estrato_value = estrato_match.group(1)
                    valor.append(estrato_value)
                    clave.append('estrato')
            
    else:
        valor.append(np.NAN)
        clave.append('estrato')
    # por ultimo el latitud y longutud
    latitud= soup.find('div',{'class':'dropdown-list light open map-section'}).find('button').get('data-x')
    longitud= soup.find('div',{'class':'dropdown-list light open map-section'}).find('button').get('data-y')
    valor.append(latitud)
    clave.append('latitud')
    valor.append(longitud)
    clave.append('longitud')
    df1 = pd.DataFrame(data=[valor],columns=clave)
    data1 = pd.concat([data1,df1],axis=0) 

In [76]:
data1

,url,habitacion,baño,dimension,precio,estrato,latitud,longitud
0,2b72-82fe-18fc293-722dc1ef005c-7129,2,2,75m2,290.000.000,5,5.532431,-73.361601
0,2688-879f-6b245957-690b01267248-3152,3,2,78m2,250.000.000,NaN,5.534,-73.362
0,9e55-b016-c7b4375e-2882ef5e1b71-43d9,3,2,56m2,170.000.000,3,5.514202,-73.368592
0,eacb-885b-18f4d5b-1661e30e9c86-7c1e,3,2,"62,32m2",262.900.000,3,5.512884,-73.367969
0,b6e8-98d2-18f4d8f-a982c7d04954-7a3c,3,2,"62,32m2",202.900.000,3,5.512952,-73.367884
...,...,...,...,...,...,...,...,...
0,eda1-adab-3f411cfe-bcf06cbe2b18-3d86,3,2,150m2,255.000.000,NaN,5.534,-73.362
0,59f1-a652-9a1311b9-257c0504a580-47d7,N,N,NAN,250.000.000,NaN,5.5144414,-73.3577575
0,e18-ba34-3c693d9b-382d6d9d9900-3416,7,6,300m2,1.800.000.000,6,5.527421,-73.361366
0,b23a-8c5f-59b874b5-d9eb0f41cb1d-4997,3,2,NAN,220.000.000,3,5.5172766,-73.3603543


In [69]:
df.iloc[i]

path              be83-9dfb-343693d2-bb81d637038f-40d2
tipo                                              Casa
estado                                           venta
ubicacion    San Ignacio/Centro Histórico/Tunja/Boyacá
Name: 77, dtype: object

In [70]:
path+'be83-9dfb-343693d2-bb81d637038f-40d2'

'https://www.puntopropiedad.com/inmueble/be83-9dfb-343693d2-bb81d637038f-40d2'

In [77]:
data1.to_csv('data2.csv',index=False)

In [57]:
a = 5
b = 0
try:
    c = a/b
except ZeroDivisionError:
    b = 1
    c = 0
b,c    

(1, 0)

In [78]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 0
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   url         136 non-null    object
 1   habitacion  136 non-null    object
 2   baño        136 non-null    object
 3   dimension   136 non-null    object
 4   precio      136 non-null    object
 5   estrato     24 non-null     object
 6   latitud     136 non-null    object
 7   longitud    136 non-null    object
dtypes: object(8)
memory usage: 9.6+ KB


In [73]:
i

77